In [ ]:
import os
import sys
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from skimage.transform import resize
import torch
import torch.nn as nn
import cv2

sys.path.append(os.path.dirname(os.path.abspath('')))
import utils.prepare_data as pd
import utils.xyzuvd as xyzuvd
import utils.visual as visual
from utils.directory import DATA_DIR, DATASET_DIR
import model as m
import loss as l
import dataset as d
import hpo_net as hnet

In [ ]:
save_prefix = 'train_fpha'
dataset = d.FPHA(save_prefix, aug=True)
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                           batch_size=1,
                                           shuffle=False,
                                           num_workers=4,
                                           sampler=None)
REORDER = visual.REORDER

In [ ]:
i = 0
idx = 0
for img, uvd_gt_no_offset, uvd_gt, hand_cell_idx in data_loader:
    if i == idx:
        uvd_gt = np.squeeze(uvd_gt)
        uvd_gt_no_offset = np.squeeze(uvd_gt_no_offset)
#         uvd_gt_no_offset[..., 0] = uvd_gt_no_offset[..., 0]*416
#         uvd_gt_no_offset[..., 1] = uvd_gt_no_offset[..., 1]*416
        img = np.squeeze(np.reshape(img, (img.shape[0], img.shape[2], img.shape[3], img.shape[1])))
        img = cv2.normalize(np.asarray(img), None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_8UC3)
        fig, ax = plt.subplots(figsize=(15, 15))
        ax.imshow(img)
        visual.visualize_joints_2d(ax, uvd_gt[REORDER], joint_idxs=False)
        visual.visualize_joints_2d(ax, uvd_gt_no_offset[REORDER], joint_idxs=False)

        index = np.where(hand_cell_idx.flatten() == 1)
        index = np.asarray(np.unravel_index(index, (13, 13, 5)))
        index = np.squeeze(index)
        import matplotlib.patches as patches
        fig, ax = plt.subplots(figsize=(13,13))
        ax.imshow(img)
        visual.visualize_joints_2d(ax, uvd_gt[REORDER], joint_idxs=False)
        for i in range(index.shape[1]):
            x = index[0, i]
            y = index[1, i]
            rect = patches.Rectangle((x*32,y*32),32,32,linewidth=1,edgecolor='r', facecolor='r', fill=True, alpha=0.5)
            ax.add_patch(rect)
        fig, ax = plt.subplots(figsize=(13,13))
        visual.visualize_joints_2d(ax, uvd_gt_no_offset[REORDER], joint_idxs=False)
        break
    i += 1